# [모듈 1.1] 모델 배포 파이프라인 개발 (SageMaker Model Building Pipeline 모든 스텝)

이 노트북은 아래와 같은 목차로 진행 됩니다. 전체를 모두 실행시에 완료 시간은 **약 15분** 소요 됩니다.

- 1. 모델 배포 파이프라인 개요
- 2. 환경 설정
- 3. 세이지 메이커 파이프라인 생성
    - (1) 모델 승인 상태 변경 람다 스텝    
    - (2) 배포할 세이지 메이커 모델 스텝 생성
    - (3) 모델 앤드 포인트 배포를 위한 람다 스텝 생성    
- 4.모델 빌딩 파이프라인 정의 및 실행
- 5.변수 저장
    
---

# 1. 모델 배포 파이프라인 개요

- 이 노트북은 다음과 같은 상황에 대한 파이프라인 입니다.
    - 모델 레제스트리에 여러개의 모델 패키지 그룹이 있습니다.
    - 모델 패키지 그룹에서 특정한 것을 선택하여 가장 최근에 저장된 모델 버전을 선택 합니다.
    - 선택된 모델 버전의 "모델 승인 상태"를 "Pending" 에서 "Approved" 로 변경 합니다.
    - 이 모델 버전에 대해서 세이지 메이커 모델을 생성합니다.
    - 세이지 메이커 모델을 기반으로 앤드포인트를 생성 합니다.

# 2. 환경 설정 



In [ ]:
%load_ext autoreload
%autoreload 2

# src 폴더 경로 설정
import sys
sys.path.append('./src')

In [ ]:
import boto3
import sagemaker
import pandas as pd

region = boto3.Session().region_name
sagemaker_session = sagemaker.session.Session()
role = sagemaker.get_execution_role()
print("role: ", role)

sm_client = boto3.client('sagemaker', region_name=region)



In [ ]:
%store -r model_package_group_name
%store -r bucket
%store -r project_prefix

In [ ]:
print("model_package_group_name: \n", model_package_group_name)
print("bucket: \n", bucket)
print("project_prefix: \n", project_prefix)

# 3. 세이지 메이커 파이프라인 생성

## 3.1. 모델 빌딩 파이프라인 변수 및 세션 생성

파이프라인에 인자로 넘길 변수는 아래 크게 3가지 종류가 있습니다.
- 모델 레지스트리에 모델 등록시에 모델 승인 상태 값    


In [ ]:
from sagemaker.workflow.parameters import (
    ParameterInteger,
    ParameterString,
    ParameterFloat,
)


model_approval_status = ParameterString(
    name="ModelApprovalStatus", default_value="PendingManualApproval"
)


## 3.2. 파이프라인 스텝 단계 정의

### 3.2.1. 모델 승인 상태 변경 람다 스텝
- 모델 레지스트리에서 해당 모델 패키지 그룹을 조회하고, 가장 최신 버전의 모델에 대해서 '모델 승인 상태 변경' 을 합니다.




In [ ]:
lambda_role = boto3.client('iam').get_role(RoleName = 'MLOps-LambdaDeploymentRole').get('Role').get('Arn')
print("lambda_role: \n", lambda_role)

In [ ]:
from sagemaker.lambda_helper import Lambda
from sagemaker.workflow.lambda_step import (
    LambdaStep,
    LambdaOutput,
    LambdaOutputTypeEnum,
)


function_name = "sagemaker-lambda-step-approve-model-deployment"

print("function_name: \n", function_name)

In [ ]:
# Lambda helper class can be used to create the Lambda function
func_approve_model = Lambda(
    function_name=function_name,
    execution_role_arn=lambda_role,
    script="src/iam_change_model_approval.py",
    handler="iam_change_model_approval.lambda_handler",
)

output_param_1 = LambdaOutput(output_name="statusCode", output_type=LambdaOutputTypeEnum.String)
output_param_2 = LambdaOutput(output_name="body", output_type=LambdaOutputTypeEnum.String)
output_param_3 = LambdaOutput(output_name="image_uri_approved", output_type=LambdaOutputTypeEnum.String)
output_param_4 = LambdaOutput(output_name="ModelDataUrl_approved", output_type=LambdaOutputTypeEnum.String)

step_approve_lambda = LambdaStep(
    name="LambdaApproveModelStep",
    lambda_func=func_approve_model,
    inputs={
        "model_package_group_name" : model_package_group_name,
        "ModelApprovalStatus": "Approved",
    },
    outputs=[output_param_1, output_param_2, output_param_3, output_param_4],
)


## 3.3 배포할 세이지 메이커 모델 스텝 생성
- 위의 람다 스텝에서 "모델 승인 상태" 를 변경한 모델에 대하여 '모델 레지스트리'에서 저장된 도커 컨테이너 이미지, 모델 아티펙트의 위치를 가져 옵니다.
- 이후에 이 두개의 인자를 가지고 세이지 메이커 모델을 생성 합니다.

In [ ]:
model_package_group_name

In [ ]:
import os
import time
from sagemaker.pytorch.model import PyTorchModel


In [ ]:
from sagemaker.workflow.pipeline_context import PipelineSession
pipeline_session = PipelineSession()    

In [ ]:
%%time 

from sagemaker.workflow.model_step import ModelStep
from sagemaker.model import Model

endpoint_name = "sm-ncf-{}".format(int(time.time()))

model = Model(
    image_uri= step_approve_lambda.properties.Outputs["image_uri_approved"],
    model_data = step_approve_lambda.properties.Outputs["ModelDataUrl_approved"],    
    role=role,
    sagemaker_session=pipeline_session,
)


In [ ]:
instance_type = 'ml.g4dn.xlarge' # $ 0.906 per Hour

step_model_create = ModelStep(
   name="MyModelCreationStep",
   step_args=model.create(instance_type = instance_type)
)

### 3.3.1. 모델 앤드 포인트 배포를 위한 람다 스텝 생성
- 람다 함수는 입력으로 세이지 메이커 모델, 앤드 포인트 컨피그 및 앤드 포인트 이름을 받아서, 앤드포인트를 생성 함.


In [ ]:
import time 

current_time = time.strftime("%m-%d-%H-%M-%S", time.localtime())

# model_name = project_prefix + "-lambda-model" + current_time
endpoint_config_name = "lambda-deploy-endpoint-config-" + current_time
endpoint_name = "lambda-deploy-endpoint-" + current_time

# function_name = "sagemaker-lambda-step-endpoint-deploy-" + current_time
function_name = "sagemaker-lambda-step-endpoint-deployment"

# print("model_name: \n", model_name)
print("endpoint_config_name: \n", endpoint_config_name)
print("endpoint_config_name: \n", len(endpoint_config_name))
print("endpoint_name: \n", endpoint_name)
print("function_name: \n", function_name)




In [ ]:
# Lambda helper class can be used to create the Lambda function
func_deploy_model = Lambda(
    function_name=function_name,
    execution_role_arn=lambda_role,
    script="src/iam_create_endpoint.py",
    handler="iam_create_endpoint.lambda_handler",
    timeout = 900, # 디폴트는 120초 임. 10분으로 연장
)

output_param_1 = LambdaOutput(output_name="statusCode", output_type=LambdaOutputTypeEnum.String)
output_param_2 = LambdaOutput(output_name="body", output_type=LambdaOutputTypeEnum.String)
output_param_3 = LambdaOutput(output_name="other_key", output_type=LambdaOutputTypeEnum.String)

step_deploy_lambda = LambdaStep(
    name="LambdaDeployStep",
    lambda_func=func_deploy_model,
    inputs={
        "model_name": step_model_create.properties.ModelName,
        "endpoint_config_name": endpoint_config_name,
        "endpoint_name": endpoint_name,
        "instance_type": instance_type,        
    },
    outputs=[output_param_1, output_param_2, output_param_3],
)

# 4.모델 빌딩 파이프라인 정의 및 실행
위에서 정의한 아래의 4개의 스텝으로 파이프라인 정의를 합니다.


In [ ]:
from sagemaker.workflow.pipeline import Pipeline



pipeline_name = 'ncf-pipeline-nb-serving'
pipeline = Pipeline(
    name=pipeline_name,
    parameters=[
        model_approval_status,        
    ],

#  steps=[step_approve_lambda, step_model_create],    
  steps=[step_approve_lambda, step_model_create, step_deploy_lambda],    
)



In [ ]:
import json

definition = json.loads(pipeline.definition())
# definition
pipeline.upsert(role_arn=role)
execution = pipeline.start()

### 파이프라인 운영: 파이프라인 대기 및 실행상태 확인

워크플로우의 실행상황을 살펴봅니다. 

실행이 완료될 때까지 기다립니다. 

In [ ]:
execution.wait()

실행된 단계들을 리스트업합니다. 파이프라인의 단계실행 서비스에 의해 시작되거나 완료된 단계를 보여줍니다.

In [ ]:
execution.list_steps()

# 5. 변수 저장

In [ ]:
depolyment_endpoint_name = endpoint_name
%store depolyment_endpoint_name

all_deployment_pipeline_name = pipeline_name
%store all_deployment_pipeline_name

# 6. SageMaker Studio 에서 파이프라인 보기
스튜디오를 사용하기 위해서는 아래 링크를 참조하여, 로그인하시기 바랍니다.
- [Amazon SageMaker Studio](https://docs.aws.amazon.com/sagemaker/latest/dg/studio.html)

## 6.1. 모델 서빙 파이프라인  보기

![sm-serving-pipeline.png](img/sm-serving-pipeline.png)

## 6.2. 모델 서빙 파이프라인 실행 내용 보기

![sm-pipeline-serving-exe.png](img/sm-pipeline-serving-exe.png)

## 6.3. 모델 서빙 파이프라인 단계 별 실행 결과 보기

![sm-pipeline-serving-steps.png](img/sm-pipeline-serving-steps.png)